<a href="https://www.kaggle.com/code/ahmedashraaff/global-warming-trends?scriptVersionId=143361687" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 align="center"><font color='Black'> Global Warming Trends from 1961 to 2022 </font></h1>

<center>
<img src="https://climate.nasa.gov/system/internal_resources/details/original/1209_shutterstock_88550854.jpg" width=500>
</center>

### <font color='289C4E'>Table of contents<font><a class='anchor' id='top'></a>
- [Installs](#installs)
- [Imports](#imports)
- [Data Preprocessing](#datapre)
- [Simple Graphs](#simple)
- [Complex Graphs](#complex)

# 1. Installs <a class='anchor' id='installs' align="center"></a> [↑](#top)

In [ ]:
!!pip install pycountry_convert
%matplotlib inline

# 2. Imports <a class='anchor' id='imports' align="center"></a> [↑](#top)

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import pycountry_convert as pc
from urllib.request import urlopen
import json
import ipywidgets as widgets
from IPython.display import display

# 3. Data Preprocessing <a class='anchor' id='datapre' align="center"></a> [↑](#top)

In [ ]:
#Load the datasets
DFLong=pd.read_csv("/kaggle/input/global-warming-trends-1961-2022/long_format_annual_surface_temp.csv")
DFWide=pd.read_csv('/kaggle/input/global-warming-trends-1961-2022/wide_format_annual_surface_temp.csv')

In [ ]:
DFLong.head()

In [ ]:
DFWide.head()

In [ ]:
#Reformat Year in DFLong
DFLong['Year']=DFLong['Year'].str.split('F', expand=True)[1]
DFLong['Year']=DFLong['Year'].astype('int64')

In [ ]:
#Rename Columns in DFWide
newcols=[str(x) for x in range(1961,2023)]
newcols.insert(0,'Country')
newcols.insert(1,'ISO2')
DFWide.columns=newcols

In [ ]:
#Rename Countries
DFLong['Country']=DFLong['Country'].str.split(',', expand=True)[0]
DFWide['Country']=DFWide['Country'].str.split(',', expand=True)[0]

In [ ]:
#Check for Null Values
display(DFWide.info())
display(DFWide.shape)

In [ ]:
#check for null values
display(DFLong.info())
display(DFLong.shape)

In [ ]:
#Check for duplicates
WideDups=DFWide.duplicated().sum()
LongDups=DFLong.duplicated().sum()
print(f'# of duplicates in DFWide: {WideDups}')
print(f'# of duplicates in DFLong: {LongDups}')

# 4. Simple Graphs <a class='anchor' id='simple' align="center"></a> [↑](#top)

In [ ]:
DFDiff=DFWide[['Country','1961','2022']].copy()
DFDiff['Change']=DFDiff['2022']-DFDiff['1961']
DFDiff=DFDiff.sort_values(by='Change',ascending=False).reset_index().drop('index',axis=1)

In [ ]:
sns.set_style("darkgrid")

fig,(ax0,ax1)=plt.subplots(nrows=1,ncols=2,figsize=(12,8))

DFDiff[:10].plot(kind='bar',x='Country',y=['1961','2022'],ax=ax0)
DFDiff[-10:].plot(kind='bar',x='Country',y=['1961','2022'],ax=ax1)

ax0.set_title("Countries with highest increase")
ax1.set_title("Countries with lowest increase")

ax0.set_ylabel("Degrees in Celsius")

plt.xticks(rotation=75)
plt.legend()
plt.show()

In [ ]:
def ISO2ToCountry(country):
    ''' Return a country's continent name from the ISO2 code
    
    Args:
        country (str): Name of the country
        
    Returns:
        str: Name of continent
    '''
    try:
        continent_code = pc.country_alpha2_to_continent_code(country)
        continent_dict = {'AF': 'Africa', 'AS': 'Asia', 'EU': 'Europe', 'NA': 'North America', 'OC': 'Oceania', 'SA': 'South America'}
        return continent_dict.get(continent_code)
    except KeyError:
        if country == 'VA':  # Vatican City
            return 'Europe'
        elif country == 'EH': # Western Sahara
            return 'Africa'

In [ ]:
DFCont = DFWide[['Country','ISO2','1961','2022']].copy()
DFCont['Continent'] = DFCont['ISO2'].apply(ISO2ToCountry)
DFCont['Change']=DFCont['2022']-DFCont['1961']

In [ ]:
conts=DFCont.groupby('Continent')['Change'].agg(np.median).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(x=conts.index,y=conts.values,ax=ax,palette='rocket')
ax.set_ylabel("Change in C")
ax.set_title("Change in temperature per continent")
plt.xticks(rotation=75)
plt.show()

In [ ]:
DFTemp=DFLong.query('Year == 2022')
DFTemp['Continent']=DFTemp['ISO2'].apply(ISO2ToCountry)

fig = px.scatter(DFTemp, x="Country", y="Temperature", color="Continent",
                 hover_data=['Country'],title='Temperature in 2022')

fig.update_layout(title_x=0.5,
                  xaxis=dict(showticklabels=False,title_text=""),
                  yaxis=dict(title_text="Degree in C"))
fig.show("notebook")

In [ ]:
DFTemp=DFLong.copy()
DFTemp['Continent']=DFTemp['ISO2'].apply(ISO2ToCountry)

fig = px.scatter(DFTemp, x="Year", y="Temperature", facet_col="Continent",facet_col_wrap=2,
                 hover_data=['Country'],title='Temperature rise per continent',trendline='ols')

fig.update_layout(title_x=0.5)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_traces(line=dict(color='red'))

fig.show("notebook")

In [ ]:
DFDecade=DFLong[['Country','Year','Temperature']]
DFDecade['Decade']=DFDecade['Year']//10 * 10

fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(data=DFDecade,x='Decade',y='Temperature',ax=ax)
ax.set_ylabel("Degree in C")
ax.set_title("Surface temperature per decade")
plt.show()

# 5. Complex Graphs <a class='anchor' id='complex' align="center"></a> [↑](#top)

In [ ]:
DFChange=DFLong.copy()
DFChange['Continent']=DFChange['ISO2'].apply(ISO2ToCountry)
DFCGroup=DFChange.groupby(['Continent','Year'])['Temperature'].agg(np.median)
DFCGroup=pd.DataFrame(DFCGroup).reset_index()

In [ ]:
fig=px.bar(DFCGroup, x="Temperature", y="Continent", animation_frame="Year", animation_group="Continent",
           color="Continent", hover_name="Continent",range_x=[-1,3],
           title='Median Temperature Per continent by year')
fig.update_layout(title_x=0.5)
fig.show("notebook")

In [ ]:
TempDF=DFWide[['Country','1961','2022','ISO2']]
TempDF['Continent']=TempDF['ISO2'].apply(ISO2ToCountry)
TempDF['Change']=abs(TempDF['2022']-TempDF['1961'])

fig = px.sunburst(TempDF, path=['Continent','Country'],values='Change',
                 title='Change in Temperature Per country')
fig.update_layout(title_x=0.5)
fig.show("notebook")

In [ ]:
#Get geoJSON data of the world
with urlopen('https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson') as response:
    countries = json.load(response)

In [ ]:
def mapfunc(year):
    """ Draw a map
    
    Args:
        df (DataFrame): DataFrame to use
        year (str): year to analyise
        
    Returns:
        fig: plotly figure
    """
    print("Please wait after selecting a different year, it takes a few seconds")
    
    fig = px.choropleth_mapbox(DFWide, geojson=countries, locations='ISO2', color=year,
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=1,labels={year:f'Temp in {year}'},
                           featureidkey="properties.ISO_A2",
                           title=f'Temperature across the world in {year}',
                           hover_name='Country')
    
    fig.update_layout(title_x=0.5)
    
    fig.show("notebook")

In [ ]:
years = sorted([str(x) for x in DFLong['Year'].unique()])

year_widget = widgets.Dropdown(options=years, description='Year:')

widgets.interact(mapfunc, year=year_widget)

<h1 align="center"><font color='Black'> Any suggestion on how to make the notebook better are welcomed😀 </font></h1>